## The `yield from` construct and its uses

`yield from` is a key construct in Python that extends the capabilities of regular `yield` within generators. It enables a powerful form of **delegation** between generators.

**How it Works:**

- When a generator (`gen`) uses `yield from` with another generator (`subgen()`), control is handed over to the sub-generator.
- `subgen()` then becomes the center of attention, yielding its values directly to the original generator's caller.

**Caller in Control:**

- The caller essentially dictates the pace of `subgen()`, iterating through it and receiving the yielded values.

**Pausing and Waiting:**

- While `subgen()` is running, the original generator (`gen`) puts its execution on hold.
- `gen` remains paused until `subgen()` finishes yielding all its values.

**Analogy:**

- Imagine `yield from` as passing the microphone. The original generator (`gen`) hands over control to `subgen()`. The recipient of the values (caller) listens directly to `subgen()` until it's done, and then control returns to `gen`.

In essence, `yield from` allows generators to collaborate seamlessly. It creates a chain where each generator can pause and yield control to another, leading to more flexible and efficient handling of complex iterators.


### The `yield from` use cases

There are several use cases that we can use `yield from` contruct.

#### `yield from` as a shortcut to `yield` in a `for` loop

One of the most basic use cases of `yield from` is using it as a shortcut to a `yield` in a `for` loop, here's an example:
```python
def gen():
    for i in range(100):
        yield i
```
This generator can be rewritten as:
```python
def gen():
    yield from range(100)
```

The `yield from x` expression in first employs the `iter(x)` function to obtain an iterator object from the operand `x`. This ensures that `x` conforms to the iterable protocol, allowing `yield from` to seamlessly integrate with diverse iterable structures. In essence, `x` can be any object that implements the iteration protocol, enabling the expression to process sequences like `lists`, `strings`, or even user-defined iterables.


#### Bidirectional Communication in Coroutines with `yield from`


The primary function of `yield from` lies in establishing a bidirectional channel between the outermost caller and the innermost subgenerator within a coroutine hierarchy. This channel facilitates the direct exchange of values (yielded back and forth) and exceptions. Notably, exceptions can be propagated all the way through the chain without the need for extensive exception handling boilerplate code in intermediate coroutines. This capability empowers coroutine delegation in a manner that was previously unattainable.

Here's a breakdown of the key points:

* **Bidirectional Channel:** `yield from` creates a two-way communication path between the outermost entity initiating the coroutine chain and the innermost subgenerator.
* **Direct Value Exchange:** Values yielded by generators can be passed directly to the caller, bypassing intermediate coroutines.
* **Exception Propagation:** Exceptions can be thrown throughout the coroutine chain and handled appropriately by the outermost caller, eliminating the need for repetitive exception handling code within each coroutine.
* **Enhanced Delegation:** This functionality fosters a powerful coroutine delegation mechanism, allowing for more modular and efficient asynchronous programming.

Before explaining this use case in detail let's define some terms:
- **Delegating generator:** The generator function that contains the `yield from <iterable>` expression.

- **Subgenerator:** The generator obtained from the `<iterable>` part of the `yield` from expression.

- **Caller:** client code that calls the delegating generator.


Within a generator function, the `yield from` expression facilitates the delegation of control flow to a subgenerator. Upon encountering `yield from subgen`, the delegating generator pauses its execution. The caller then gains the ability to transmit data directly to the subgenerator. The subgenerator, in turn, yields values back to the caller. The delegating generator resumes operation only when the subgenerator returns. If we assume the subgenerator returns a value `x`, the `yield from` expression automatically extracts this value from the `StopIteration` exception raised by the `return x` statement within the subgenerator. In simpler terms, `yield from` streamlines the process of retrieving the return value from the subgenerator's `StopIteration` exception.

Furthermore, yield from offers transparent delegation beyond just yielded values. It also propagates calls to the `.close()` and `.throw()` methods of from caller to subgenerator, ensuring proper resource management and exception handling throughout the delegation chain.



More formally, the `RESULT = yield from EXPR` is equivalent to:
```python
_i = iter(EXPR)
try:
    _y = next(_i)
except StopIteration as _e:
    _r = _e.value
else:
    while True:
        try:
            _s = yield _y
        except GeneratorExit as _e:
            try:
                _m = _i.close
            except AttributeError:
                pass
            else:
                _m()
            raise _e
        except BaseException as _e:
            _x = sys.exc_info()
            try:
                _m = _i.throw
            except AttributeError:
                raise _e
            else:
                try:
                    _y = _m(*_x)
                except StopIteration as _e:
                    _r = _e.value
                    break
        else:
            try:
                if _s is None:
                    _y = next(_i)
                else:
                    _y = _i.send(_s)
            except StopIteration as _e:
                _r = _e.value
                break
RESULT = _r
```

While the preceding code snippet might appear complex initially, it essentially implements the concepts described earlier in a more concise manner.